https://medium.com/qu4nt/extracci%C3%B3n-autom%C3%A1tica-de-frases-y-palabras-relevantes-en-espa%C3%B1ol-con-python-stanfordnlp-e0d3be5cb29d

tokenize: Tokenizador. Separa un texto en oraciones, y las oraciones en palabras.<br>
mwt: Tokenizador de palabras compuestas. Reconoce palabras como follow up o asistente virtual.<br>
pos: Clasificador de clase de palabra. Asigna categorías tales como adjetivo o verbo a una palabra.<br>
lemma: Lematizador. Toma una palabra y la transforma a su forma de diccionario. Por ejemplo, reconoce que la forma de diccionario de cantábamos es cantar.<br>
depparse: Analizador de dependencias. Señala la función de una palabra e indica cuál es su relación con otras palabras de la misma oración. Puede indicar, por ejemplo, que en la oración amo el chocolate, la frase el chocolate es el objeto amado por la persona que habla.<br>

In [66]:
import numpy as np
import pandas as pd
import string
import re
from IPython.core.display import HTML

# Gráficos
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Preprocesado y modelado
from sklearn import svm # clasificador support vector machines
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import nltk # natural lenguaje tool kit para procesamiento de lenguaje natural

# cambiador de formatos para factorizar un dataset
from sklearn.feature_extraction.text import TfidfVectorizer 



# Configuración warnings
import warnings
warnings.filterwarnings('ignore')

In [67]:
# %pip install stanza


In [68]:
# %pip install --upgrade typing_extensions


In [69]:
import stanza
# stanza.download('es')


In [70]:
def limpiar_tokenizar(texto):
    # Eliminación de números
    nuevo_texto = re.sub("\d+", ' ', texto)
    # Se convierte todo el texto a minúsculas
    nuevo_texto = nuevo_texto.lower()
    # Eliminación de páginas web (palabras que empiezan por "http")
    nuevo_texto = re.sub('http\S+', ' ', nuevo_texto)
    # Eliminación de signos de puntuación
    regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\
    \-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~—]'
    nuevo_texto = re.sub(regex , ' ', nuevo_texto)
    # Eliminación de espacios en blanco múltiples
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
    # Tokenización por palabras individuales
    nuevo_texto = nuevo_texto.split(sep = ' ')
    # Eliminación de tokens con una longitud < 2
    nuevo_texto = [token for token in nuevo_texto if len(token) > 1]
    # quitar los token que sean solamente de una letra repetidas 2 o mas veces
    nuevo_texto = [token for token in nuevo_texto if not re.fullmatch(r'(.)\1*', token)]
    # une los tokens en un solo string
    nuevo_texto = ' '.join(nuevo_texto)
    
    return(nuevo_texto)

In [71]:
import stanza
nlp = stanza.Pipeline(lang='es')
cadena_cats = "deberían modernizar los equipos utilizados en los laboratorios."
cadena_cats=limpiar_tokenizar(cadena_cats)
doc = nlp(cadena_cats)

# Imprimir las palabras y sus lemas
for sent in doc.sentences:
    for word in sent.words:
        print(f'Palabra: {word.text}\tLema: {word.lemma}')

2024-06-09 13:23:23 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-06-09 13:23:23 INFO: Downloaded file to C:\Users\Sheen\stanza_resources\resources.json
2024-06-09 13:23:25 INFO: Loading these models for language: es (Spanish):
| Processor    | Package         |
----------------------------------
| tokenize     | ancora          |
| mwt          | ancora          |
| pos          | ancora_charlm   |
| lemma        | ancora_nocharlm |
| constituency | combined_charlm |
| depparse     | ancora_charlm   |
| sentiment    | tass2020_charlm |
| ner          | conll02         |

2024-06-09 13:23:25 INFO: Using device: cpu
2024-06-09 13:23:25 INFO: Loading: tokenize
2024-06-09 13:23:25 INFO: Loading: mwt
2024-06-09 13:23:25 INFO: Loading: pos
2024-06-09 13:23:25 INFO: Loading: lemma
2024-06-09 13:23:26 INFO: Loading: constituency
2024-06-09 13

Palabra: deberían	Lema: deber
Palabra: modernizar	Lema: modernizar
Palabra: los	Lema: el
Palabra: equipos	Lema: equipo
Palabra: utilizados	Lema: utilizado
Palabra: en	Lema: en
Palabra: los	Lema: el
Palabra: laboratorios	Lema: laboratorio


In [72]:
doc.sentences[0].print_dependencies()

('deberían', 2, 'aux')
('modernizar', 0, 'root')
('los', 4, 'det')
('equipos', 2, 'obj')
('utilizados', 4, 'amod')
('en', 8, 'case')
('los', 8, 'det')
('laboratorios', 5, 'obl')


(‘Cats’, dependo de la palabra 4, y es a quien responsabilizo de lo que voy a decir porque soy un Sujeto)<br>
(‘fue’, dependo de la palabra 4, y soy un verbo copulativo — solo estoy aquí para unir cosas — )<br>
(‘una’, dependo de la palabra 4, y soy un determinante — así que no me hagas mucho caso — )<br>
(‘película’, soy la palabra más importante acá y no dependo de nadie — soy la 0 — , y soy el root o núcleo de esta oración)<br>
(‘realmente’, dependo de la palabra 6 — de ‘terrible’ — y soy un adverbio modificador)<br>
(‘terrible’, dependo de la palabra 4, y soy un adjetivo que modifica a esa palabra)<br>
(‘.’, dependo de 4, y soy un punto — no le importo a nadie en realidad — )

In [73]:
review = """Sin embargo, no las tiene todas consigo Zombie y es, precisamente, cuando tiene que lograr mantener la tensión tras el impetuoso 
retorno cuando patina. El director pierde el pulso vibrante para dejarse llevar por una trama más apagada, sin chispa y del todo previsible. 
El segundo tercio del film se desploma. Ese montaje paralelo donde vemos, de un lado a Strode intentando reponerse y a Myers resucitando de sus 
cenizas y deambulando cual sombra por el campo en un dilatado regreso a Haddonfield, se desinfla. Previsible, aburrido, sin pulso. Y dejando 
entrever las costuras de un guión en el que su creador no logra sostener el ritmo. Las pesadillas de Strode y los asesinatos de Myers se vuelven 
aburridos. Y no solo por repetición insulsa sino también porque Zombie se desata con escenas más creativas, pero carentes de originalidad. Donde 
ese doble juego entre realidad y ensoñación se vuelve insistente, pierde sutileza para convertirse en una evidente falta de recursos e 
imaginación. Aquí debería haber dejado su impronta y solo consigue rozar lo convencional, incluso lo ridículo. Basta comprobar las apariciones 
del Dr. Loomis (Malcolm McDowell) que rozan lo cómico o la insistencia con la que nos quiere recordar ese sueño de Myers con angelical madre y 
el caballo blanco, que acaban tomando un protagonismo excesivo y perdiendo toda la sugerencia que debería acompañar para convertirse en una 
incómoda y repetida presencia que acaba entorpeciendo el avance de la historia en varias escenas. Lamentablemente el desagisado hace llegar al 
tramo final sin muchas esperanzas de sorpresa. Y menos aún de tensión. El largo regreso de Myers y el desenlace ya no recuperan la brillantez 
inicial. Y muy al contrario demuestra que quizás Zombie dispuso de una gran oportunidad para reivindicarse pero que no supo aprovechar."""

In [74]:
doc_review = nlp(review)

for sent in doc_review.sentences: # recorremos cada oración
    for dep in sent.dependencies: # recorremos cada palabra
        if dep[1] == 'amod': # el subíndice 1 indica que se trata de la clase funcional
            print(dep[0].text, dep[2].text) # uso del atributo .text

retorno impetuoso
pulso vibrante
trama apagada
tercio segundo
montaje paralelo
regreso dilatado
repetición insulsa
escenas creativas
juego doble
falta evidente
convencional ridículo
madre angelical
caballo blanco
protagonismo excesivo
presencia incómoda
tramo final
regreso largo
brillantez inicial
oportunidad gran


In [75]:
# import stanza

# # Configurar stanza para el idioma español
# stanza.download('es')
# nlp = stanza.Pipeline('es')

# Array de 5 oraciones de ejemplo
oraciones = [
    "satisfecho con todo.",
    "Modernidad en los equipos e instrumentos en los laboratorios y talleres: muchos de los instrumentos no están acorde con la lo que vende en el mercado actualmente.",
    "mejoramiento en la calidad de la internet y el acceso a mas amplios dentro de las instalaciones.",
    "pocas charlas o seminarios de innovación de información enfocado para el grupo nocturno.",
    "Internet (WiFi Institucional). El internet debe de ser mejor debido a que pasamos más tiempo en la U que en otro lugar, y siempre estamos realizando investigaciones y tareas."
]

# Función para extraer la palabra núcleo de una oración
def obtener_nucleo(oracion):
    doc = nlp(oracion)
    for sent in doc.sentences:
        # Buscar la palabra núcleo (root) en la oración
        for word in sent.words:
            if word.head == 0:  # Si el head es 0, es el núcleo
                return word.text

# Iterar sobre las oraciones y mostrar el núcleo de cada una
for oracion in oraciones:
    nucleo = obtener_nucleo(oracion)
    print(f"Núcleo de '{oracion}': {nucleo}")


Núcleo de 'satisfecho con todo.': satisfecho
Núcleo de 'Modernidad en los equipos e instrumentos en los laboratorios y talleres: muchos de los instrumentos no están acorde con la lo que vende en el mercado actualmente.': Modernidad
Núcleo de 'mejoramiento en la calidad de la internet y el acceso a mas amplios dentro de las instalaciones.': mejoramiento
Núcleo de 'pocas charlas o seminarios de innovación de información enfocado para el grupo nocturno.': char
Núcleo de 'Internet (WiFi Institucional). El internet debe de ser mejor debido a que pasamos más tiempo en la U que en otro lugar, y siempre estamos realizando investigaciones y tareas.': Internet
